In [2]:
import os
import torch
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

current_dir = os.getcwd()

data_dir = os.path.join(current_dir, "..", "data")
data_path = os.path.join(data_dir, "300_data_pop.xlsx")
model_dir = os.path.join(current_dir, "..", "model")

print("Data path:", data_path)

def calculate_sum_score(df):
    df['score'] = df['creditability'] + df['content'] + df['expression']
    return df
    
def norm_score(data):
    score_min = data['score'].min()
    score_max = data['score'].max()
    data['score_norm'] = (data['score'] - score_min) / (score_max - score_min)
    return data

# Load the data
data = pd.read_excel(data_path)
data = calculate_sum_score(data)
data = norm_score(data)
data = data[['comment', 'score_norm']]

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

Data path: C:\Users\Pop\Documents\GitHub\utcc_independent_study\training\..\data\300_data_pop.xlsx


In [4]:
model_name = "airesearch/wangchanberta-base-att-spm-uncased"
config = AutoConfig.from_pretrained(model_name)
config.num_labels = 1 # regression
model1 = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
tokenizer1 = AutoTokenizer.from_pretrained(model_name)

model_name = "pythainlp/thainer-corpus-v2-base-model"
config = AutoConfig.from_pretrained(model_name)
config.num_labels = 1 # regression
model2 = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
tokenizer2 = AutoTokenizer.from_pretrained(model_name)

model_name = "KoichiYasuoka/roberta-base-thai-spm-upos"
config = AutoConfig.from_pretrained(model_name)
config.num_labels = 1 # regression
model3 = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
tokenizer3 = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wa

Some weights of the model checkpoint at pythainlp/thainer-corpus-v2-base-model were not used when initializing CamembertForSequenceClassification: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at pythainlp/thainer-corpus-v2-base-model and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this 

Some weights of the model checkpoint at KoichiYasuoka/roberta-base-thai-spm-upos were not used when initializing RobertaForSequenceClassification: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at KoichiYasuoka/roberta-base-thai-spm-upos and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this mode

airesearch/wangchanberta-base-att-spm-uncased

In [6]:
model = model1
tokenizer = tokenizer1

# Define your loss function
loss_fn = torch.nn.MSELoss()  # Change loss function to L1Loss for regression

# Move your model to the device (CPU)
device = torch.device("cpu")
model.to(device)

# Define your optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Set up gradient accumulation
gradient_accumulation_steps = 4

# Define batch and epochs
batch_size = 16  # Reduced batch size
num_epochs = 3

# Tokenize data and create dataloader
encoded_data = tokenizer.batch_encode_plus(
    train_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=200,
    return_tensors='pt'
)

labels = torch.tensor(train_data['score_norm'].values, dtype=torch.float32)  # Convert labels to float32
dataset = TensorDataset(encoded_data['input_ids'], encoded_data['attention_mask'], labels)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Fine-tune the model
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    step = 0

    for batch in train_dataloader:
        step += 1
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps

        loss.backward()

        if step % gradient_accumulation_steps == 0:
            optimizer.step()
            epoch_loss += loss.item()

    print(f'Epoch: {epoch + 1}, Loss: {epoch_loss / step}')

# model.save_pretrained(model_dir)
# tokenizer.save_pretrained(model_dir)

# Preprocess the test data
encoded_test_data = tokenizer.batch_encode_plus(
    test_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

test_labels = torch.tensor(test_data['score_norm'].values, dtype=torch.float32)
test_dataset = TensorDataset(encoded_test_data['input_ids'], encoded_test_data['attention_mask'], test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Evaluate the model's predictions against the actual values
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = logits.squeeze(1).cpu().numpy()

        predictions.extend(preds)
        actuals.extend(labels.cpu().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

mse = mean_squared_error(actuals, predictions)
mae = mean_absolute_error(actuals, predictions)
print(f'Mean Squared Error: {mse:.4f}')
print(f'Mean Absolute Error: {mae:.4f}')

result_df = pd.DataFrame({'actual': actuals, 'predicted': predictions})

form = pd.DataFrame(columns=['comment', 'actual', 'predicted'])
count = 0
for i in zip(test_data['comment'], result_df['actual'], result_df['predicted']):
    form.loc[count] = [i[0], i[1], i[2]]
    count += 1

sorted_df = form.sort_values(by='predicted', ascending=False)

for index, row in sorted_df.iterrows():
    print(f"comment: {row['comment']}")
    print(f"actual: {row['actual']}")
    print(f"predicted: {row['predicted']}")
    print('')

Epoch: 1, Loss: 0.019131021574139595
Epoch: 2, Loss: 0.009090022067539394
Epoch: 3, Loss: 0.009296075208112597
Mean Squared Error: 0.0564
Mean Absolute Error: 0.2018
comment: ได้รับสินค้าแล้วว ส่งก่อนกำหนด4วันเลย สินค้าตรงตามปก ราคาดีไม่แพง เคยใช้แล้วชอบ รู้สึกป้องกันลิปเปื้อนแมสได้ดี
actual: 0.8888888955116272
predicted: 1.1526739597320557

comment: จัดส่งไวมากค่ะ ราคาก็ถูกดี เชื่อมต่อง่ายมาก เวลาใช้ค่อนข้างลื่น ไวค่ะ ก็ถนัดดี ไว้จะอุดหนุนใหม่นะคะ
actual: 0.7777777910232544
predicted: 1.1065387725830078

comment: จัดส่งเร็วค่ะ ของได้ตรงรูปตรงปกทั้งเมาส์ทั้งแป้นพิมพ์ ส่วนตัวรู้สึกว่าแป้นพิมพ์บางไปหน่อยแต่ก็สมกับราคานี้ค่ะ มาที่เมาส์แนะนำว่าให้หาแผ่นรองเมาส์ด้วยนะคะสำหรับใครที่ไม่มี เพราะถ้าใช้กับโต๊ะเปล่าๆเลเซอร์จะไปติดค่ะ โดยรวมถือว่าดีคุ้มค่าทั้งราคาและระยะเวลาการส่งของค่ะ😊
actual: 0.7777777910232544
predicted: 1.0861258506774902

comment: คุณภาพของสินค้าดีมาก ความคุ้มค่าดีมาก ความเร็วในการจัดส่งดีมากๆ การให้บริการจากร้านค้าดีมาก ถูกใจมากๆ ทุกอย่าง✨
actual: 0.8888888955116272
predict

pythainlp/thainer-corpus-v2-base-model

In [7]:
model = model2
tokenizer = tokenizer2

# Define your loss function
loss_fn = torch.nn.MSELoss()  # Change loss function to L1Loss for regression

# Move your model to the device (CPU)
device = torch.device("cpu")
model.to(device)

# Define your optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Set up gradient accumulation
gradient_accumulation_steps = 4

# Define batch and epochs
batch_size = 16  # Reduced batch size
num_epochs = 3

# Tokenize data and create dataloader
encoded_data = tokenizer.batch_encode_plus(
    train_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=200,
    return_tensors='pt'
)

labels = torch.tensor(train_data['score_norm'].values, dtype=torch.float32)  # Convert labels to float32
dataset = TensorDataset(encoded_data['input_ids'], encoded_data['attention_mask'], labels)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Fine-tune the model
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    step = 0

    for batch in train_dataloader:
        step += 1
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps

        loss.backward()

        if step % gradient_accumulation_steps == 0:
            optimizer.step()
            epoch_loss += loss.item()

    print(f'Epoch: {epoch + 1}, Loss: {epoch_loss / step}')

# model.save_pretrained(model_dir)
# tokenizer.save_pretrained(model_dir)

# Preprocess the test data
encoded_test_data = tokenizer.batch_encode_plus(
    test_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

test_labels = torch.tensor(test_data['score_norm'].values, dtype=torch.float32)
test_dataset = TensorDataset(encoded_test_data['input_ids'], encoded_test_data['attention_mask'], test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Evaluate the model's predictions against the actual values
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = logits.squeeze(1).cpu().numpy()

        predictions.extend(preds)
        actuals.extend(labels.cpu().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

mse = mean_squared_error(actuals, predictions)
mae = mean_absolute_error(actuals, predictions)
print(f'Mean Squared Error: {mse:.4f}')
print(f'Mean Absolute Error: {mae:.4f}')

result_df = pd.DataFrame({'actual': actuals, 'predicted': predictions})

form = pd.DataFrame(columns=['comment', 'actual', 'predicted'])
count = 0
for i in zip(test_data['comment'], result_df['actual'], result_df['predicted']):
    form.loc[count] = [i[0], i[1], i[2]]
    count += 1

sorted_df = form.sort_values(by='predicted', ascending=False)

for index, row in sorted_df.iterrows():
    print(f"comment: {row['comment']}")
    print(f"actual: {row['actual']}")
    print(f"predicted: {row['predicted']}")
    print('')

Epoch: 1, Loss: 0.024240271653980017
Epoch: 2, Loss: 0.007770168886054307
Epoch: 3, Loss: 0.0061607189127244055
Mean Squared Error: 0.0665
Mean Absolute Error: 0.2102
comment: ได้รับสินค้าแล้วว ส่งก่อนกำหนด4วันเลย สินค้าตรงตามปก ราคาดีไม่แพง เคยใช้แล้วชอบ รู้สึกป้องกันลิปเปื้อนแมสได้ดี
actual: 0.8888888955116272
predicted: 1.137207269668579

comment: ส่งสินค้ารวดเร็วทันใจ ได้สินค้าตรตามที่สั่ง แพ็คเกจมีความเสียหายเล็กน้อยแตไม่มีผลกับสินค้า ราคาเหมาะสม โดยรวมแล้วถูกใจครับผม โอกาสหน้าค่อยซื้อเพิ่มนะครับ
actual: 0.8888888955116272
predicted: 1.0993088483810425

comment: ได้รับของครบถ้วนค่ะ การจัดส่งไว สินค้าดูดี ราคาไม่แพง คุ้มค่ามากๆ ครั้งต่อไปสั่งอีกแน่นอนค่ะ สายสวยๆ น่ารักๆ
actual: 0.7777777910232544
predicted: 1.0682270526885986

comment: ได้รับสินค้าเร็วมากภายในไม่กี่วัน เนื่องจากส่งจากในไทยเลย พัสดุกล่อง ยังไม่ได้แกะซองของสินค้า แต่คาดว่าน่าจะใช้งานได้ปกติ ไม่น่ามีปัญหาอะไร ประทับใจและจะกลับมาซื้ออีกครับ
actual: 1.0
predicted: 1.060790777206421

comment: สินค้าส่งมารวดเร็วทันใจดีมาก

KoichiYasuoka/roberta-base-thai-spm-upos

In [8]:
model = model3
tokenizer = tokenizer3

# Define your loss function
loss_fn = torch.nn.MSELoss()  # Change loss function to L1Loss for regression

# Move your model to the device (CPU)
device = torch.device("cpu")
model.to(device)

# Define your optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Set up gradient accumulation
gradient_accumulation_steps = 4

# Define batch and epochs
batch_size = 16  # Reduced batch size
num_epochs = 3

# Tokenize data and create dataloader
encoded_data = tokenizer.batch_encode_plus(
    train_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=200,
    return_tensors='pt'
)

labels = torch.tensor(train_data['score_norm'].values, dtype=torch.float32)  # Convert labels to float32
dataset = TensorDataset(encoded_data['input_ids'], encoded_data['attention_mask'], labels)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Fine-tune the model
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    step = 0

    for batch in train_dataloader:
        step += 1
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps

        loss.backward()

        if step % gradient_accumulation_steps == 0:
            optimizer.step()
            epoch_loss += loss.item()

    print(f'Epoch: {epoch + 1}, Loss: {epoch_loss / step}')

# model.save_pretrained(model_dir)
# tokenizer.save_pretrained(model_dir)

# Preprocess the test data
encoded_test_data = tokenizer.batch_encode_plus(
    test_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

test_labels = torch.tensor(test_data['score_norm'].values, dtype=torch.float32)
test_dataset = TensorDataset(encoded_test_data['input_ids'], encoded_test_data['attention_mask'], test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Evaluate the model's predictions against the actual values
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = logits.squeeze(1).cpu().numpy()

        predictions.extend(preds)
        actuals.extend(labels.cpu().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

mse = mean_squared_error(actuals, predictions)
mae = mean_absolute_error(actuals, predictions)
print(f'Mean Squared Error: {mse:.4f}')
print(f'Mean Absolute Error: {mae:.4f}')

result_df = pd.DataFrame({'actual': actuals, 'predicted': predictions})

form = pd.DataFrame(columns=['comment', 'actual', 'predicted'])
count = 0
for i in zip(test_data['comment'], result_df['actual'], result_df['predicted']):
    form.loc[count] = [i[0], i[1], i[2]]
    count += 1

sorted_df = form.sort_values(by='predicted', ascending=False)

for index, row in sorted_df.iterrows():
    print(f"comment: {row['comment']}")
    print(f"actual: {row['actual']}")
    print(f"predicted: {row['predicted']}")
    print('')

Epoch: 1, Loss: 0.02072158391820267
Epoch: 2, Loss: 0.0054933749488554895
Epoch: 3, Loss: 0.00597289192955941
Mean Squared Error: 0.0318
Mean Absolute Error: 0.1460
comment: จัดส่งเร็วค่ะ ของได้ตรงรูปตรงปกทั้งเมาส์ทั้งแป้นพิมพ์ ส่วนตัวรู้สึกว่าแป้นพิมพ์บางไปหน่อยแต่ก็สมกับราคานี้ค่ะ มาที่เมาส์แนะนำว่าให้หาแผ่นรองเมาส์ด้วยนะคะสำหรับใครที่ไม่มี เพราะถ้าใช้กับโต๊ะเปล่าๆเลเซอร์จะไปติดค่ะ โดยรวมถือว่าดีคุ้มค่าทั้งราคาและระยะเวลาการส่งของค่ะ😊
actual: 0.7777777910232544
predicted: 0.8775432705879211

comment: สินค้ามาตรงปกค่ะ ลูกชอบมากค่ะมีสีสันสวยเวลาพิมพ์ ก็มีสีหลายสี ให้เลือก เวลาพิมพ์ง่ายไม่มีปัญหาในการใช้แป้นพิมพ์ค่ะ
actual: 0.6666666865348816
predicted: 0.8536656498908997

comment: สินค้าส่งมารวดเร็วทันใจดีมากค่ะ สินค้าส่งมาเรียบร้อยดีมากค่ะ สินค้าราคาถูกดีมากค่ะ สินค้าดีคุ้มค่ามากค่ะ
actual: 0.7777777910232544
predicted: 0.8508191108703613

comment: ชอบมากเลยดีไซน์เอยสีเอ่ยดีไปหมดดูเรียบแต่สวยสีดำสวยมากสีเทาก็สวยไว้จะสั่งสีเทาอีกค่ะทรงสวยเสียงดีไม่เจ็บหู
actual: 0.7777777910232544
pred

In [ ]:
test_data = pd.DataFrame({
    'comment': [
        'คงโชคร้ายเอง ฟังได้ข้างเดียวหรือเชื่อมต่อไม่เป็นก็ยังงง อีกข้างเสียงดังชัดเจน เอาไปเชื่อมอีกเครื่องก็ใช้ได้ข้างเดียวเหมือนกัน',
        'จากที่ลองใช้มาสักพักดีมากเสียงดีและชัดเสียงไม่แตกเบสหนักฟังเพลงเสียงดีมากมีตำหนินิดหน่อยตรงขอบแต่ไม่เป็นไรแบตทนมากชาร์จ1ครั้งตั้งแต่ได้มาวันแรกจนถึงวันนี้แบตยังเหลืออีก50เปอร์เซ็นต์',
        'คุณภาพแย่ชาร์จไม่ได้และไม่ชดใช้คืน',
        'มีรอยร้าว',
        'ต้องชารจ์แบตก่อนเปิด เปิดแล้วฟังเพลงได้ดี ใส่การ์ดแล้วสามารถเล่นได้ปุ่มกด งงๆ นิดหน่อย ปุ่มเดียวกัน กดสั้น กดยาว ให้ผลต่างกัน',
        'ได้รับสินค้าเรียบร้อยครับ​ ส่งเร็วดี​ ราคาไม่แพง​ ใครสนใจสั่งซื้อได้เลยครับ​ ยังไม่ได้ลองใช้แต่คิดว่าคงไม่มีปัญหา'
    ],
    'score': [5, 7, 6, 4, 5, 6]
})

OSError: pythainlp/ulmfit-base-thai is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.